## TLE Parser
* https://pypi.org/project/ephem/ 
* https://federicostra.github.io/tletools/ 

In [1]:
# from IPython.display import Image, HTML
import numpy as np
import pandas as pd
import geopy.distance
import math
#import time
import datetime
import ephem
from math import degrees

import warnings

warnings.filterwarnings("ignore")           # Suppress Warning

In [2]:
WorkingFolder = "/Users/cv0361/Desktop/TechChallenge/Data/"

In [3]:
# Sample Success Parse
line1 = "ISS (ZARYA)"
line2 = "1 25544U 98067A   03097.78853147  .00021906  00000-0  28403-3 0  8652"
line3 = "2 25544  51.6361  13.7980 0004256  35.6671  59.2566 15.58778559250029"

iss = ephem.readtle(line1, line2, line3)

print("SatelliteId: {}\tTLE Epoch: {}".format(iss.catalog_number, iss._epoch))

iss.compute(iss._epoch)
print("Lat: {}\tLong: {}".format(degrees(iss.sublat), degrees(iss.sublong)))

iss.compute('2003/4/7 19:00')
print("Lat: {}\tLong: {}".format(degrees(iss.sublat), degrees(iss.sublong)))


SatelliteId: 25544	TLE Epoch: 2003/4/7 18:55:29
Lat: 51.34448200999389	Long: -7.731682786674285
Lat: 46.35361497214138	Long: 16.99506793812249


In [6]:
# # Data causing parsing error
# line1 = "NULL"
# line2 = "1 41857U 16068A   17001.92200105 -.00000043 +00000-0 +00000-0 0 0173"
# line3 = "2 41857 098.5098 004.1411 0012813 111.3144 248.9419 14.3270115000730"

# iss = ephem.readtle(line1, line2, line3)


In [7]:
filename = WorkingFolder + "TLE/tle2017.txt"

with open(filename, 'r') as f:
    Lines = f.readlines()

print("Total lines:", len(Lines))

Total lines: 15345762


In [8]:
Header = ['SatNum', 'Class', 'InternDes', 'Epoch', 'Ballistic', 'MeanMotion', 'DragTerm', 'Ephemeris', 'ElementNum', 
          'Inclination', 'RightAsc', 'Eccentricity', 'ArgPerigee', 'MeanAno', 'MeanMot', 'Lat', 'Lon', 'EpochDate', 'errTLE']

def parseTLE(line1, line2, line3):
    x = line2.split()
    SatNum = x[1][:-1]
    Classi = x[1][-1:]
    return [SatNum, Classi] + x[2:] + line3.split()[2:8]

# line1 = "ISS (ZARYA)"
# line2 = "1 25544U 98067A   03097.78853147  .00021906  00000-0  28403-3 0  8652"
# line3 = "2 25544  51.6361  13.7980 0004256  35.6671  59.2566 15.58778559250029"
# parseTLE(line1, line2, line3)

In [9]:
count = 0
line1 = "NULL" 
data_array = list()

try:
    for line in Lines: 
        count += 1

        if count%2:
            line2 = line.strip().replace("\\", "")

        else:
            line3 = line.strip()

#             if "41857U" not in line2: continue   # Filter by specific satellite

            try:
                iss = ephem.readtle(line1, line2, line3)
            except:
#                 print("*********** Parsing Error")
                data_array.append(parseTLE(line1, line2, line3) + [None, None, '', 1])
                continue

            iss.compute(iss._epoch)
            lat = round(degrees(iss.sublat), 6)
            lon = round(degrees(iss.sublong), 6)
            
            data_array.append(parseTLE(line1, line2, line3) + [lat, lon, str(iss._epoch), 0])
            
#             print(line2)
#             print(line3)
#             print("SatId: {}\tEpoch: {}\tLat: {}\tLong: {}".format(iss.catalog_number, iss._epoch, lat, lon))
#             print()

#         if count > 7: break   # Get off early
except:
    pass

print("Line:", count)

df = pd.DataFrame(data_array, columns=Header)
df.head()

Line: 1168894


,SatNum,Class,InternDes,Epoch,Ballistic,MeanMotion,DragTerm,Ephemeris,ElementNum,Inclination,RightAsc,Eccentricity,ArgPerigee,MeanAno,MeanMot,Lat,Lon,EpochDate,errTLE
0,41857,U,16068A,16366.94427814,-.00000043,00000-0,00000+0,0,1682,98.5102,3.1811,0012704,113.7213,246.5323,14.32700070,0.001747,-77.541942,2016/12/31 22:39:46,0
1,41909,U,16083C,16366.95089246,-.00000722,22793-5,00000+0,0,222,97.5945,78.7102,0220259,352.7158,7.6030,15.68490028,0.529794,-4.470950,2016/12/31 22:49:17,0
2,41844,U,16066D,17001.00057939,-.00000070,00000-0,00000+0,0,1823,97.4050,10.7457,0018148,104.4001,338.9419,15.21567598,79.991877,-137.775653,2017/1/1 00:00:50,0
3,25544,U,98067A,16366.94039976,.00001110,00000-0,24188-4,0,4578,51.6423,152.0418,0007056,39.1392,105.7402,15.53963830,26.696288,-130.749683,2016/12/31 22:34:11,0
4,41857,U,16068A,17001.08394256,-.00000043,00000-0,00000+0,0,1698,98.5100,3.3185,0012675,113.2709,246.9829,14.32700431,0.001796,-127.821403,2017/1/1 02:00:53,0


In [10]:
df.loc[df.errTLE == 1].head()

,SatNum,Class,InternDes,Epoch,Ballistic,MeanMotion,DragTerm,Ephemeris,ElementNum,Inclination,RightAsc,Eccentricity,ArgPerigee,MeanAno,MeanMot,Lat,Lon,EpochDate,errTLE
6294,5,U,58002B,16366.75742498,-.00000016,+00000-0,+64253-5,0,0001,034.2413,160.5515,1845077,066.4765,311.8522,10.8470961306801,NaN,NaN,,1
6295,11,U,59001A,16366.76980551,+.00000154,+00000-0,+71303-4,0,0001,032.8741,118.7662,1469384,319.2317,030.6912,11.8540987312496,NaN,NaN,,1
6296,12,U,59001B,16366.60456233,+.00000388,+00000-0,+20765-3,0,0001,032.9127,092.2463,1668268,272.2241,068.9101,11.4409385013518,NaN,NaN,,1
6297,16,U,58002A,16366.91013561,-.00000036,+00000-0,-62610-4,0,0001,034.2700,202.8836,2028614,310.3542,033.6450,10.4867152232618,NaN,NaN,,1
6298,20,U,59007A,16366.37828625,+.00000245,+00000-0,+10661-3,0,0001,033.3454,004.3618,1668352,042.6413,329.2272,11.5539158508434,NaN,NaN,,1


In [11]:
# Output result to csv
df.to_csv(WorkingFolder + "TLE/tle2017.csv", index=False)